![NVIDIA Logo](images/nvidia.png)

# P-tune on PubMedQA

In this notebook we p-tune 3 NeMo GPT models for the PubMedQA question answering task.

---

## Learning Objectives

By the time you complete this notebook you will be able to:
- Know how to create a p-tuning customization using NeMo Service.

---

## Mocking the NeMo Service

As mentioned in the previous notebook there are 2 places in this workshop where we will you will be interacting with mocks of the NeMo Service. The first was for file management, and the second is for running customization jobs like p-tuning and later, LoRA. We will still be confirming that you launch the customizations correctly.

By mocking customization creation we can provide you with immediate access to customizations we have already created using the exact same process described in the workshop (exact same model, exact same data, exact same configurations) without you needing to wait for the customization jobs to complete which depending on the customization job in question is roughly between 5 and 30 minutes.

---

## Set IDs for Training and Validation Data

In the previous notebook you mock uploaded training and validation data and in the response from `upload` you received unique identifiers for the files. Typically you would capture those IDs to supply to your customization job, but in our case we will simply provide you with IDs here.

In [ ]:
training_dataset_file_id = '6db1cb59-de77-426c-afb7-14220c0fee73' # ID obtained from file upload to NeMo Service in previous step
validate_dataset_file_id = '4c5b9022-95d2-40a2-95d5-75115f6e7efb' # ID obtained from file upload to NeMo Service in previous step

---

## Create Customization (P-tuned Model)

With the data uploaded and well-formatted, p-tuning with NeMo Service is incredibly simple, all we have to do is use the `conn.create_customization` method.

### Create Customization Method

Let's look at the docstring for `conn.create_customization` to better understand how we should use it.

```python
Help on method create_customization in module nemollm.api:

create_customization method of nemollm.api.NemoLLM instance
    - model: str
    - training_dataset_file_id: str
    - name: str
    - description: Optional[str] = None
    - training_type: Optional[str] = None
    - shared_with: Optional[List[str]] = None
    - validation_dataset_file_id: Optional[str] = None
    - batch_size: Optional[int] = None
    - epochs: Optional[int] = None
    - learning_rate: Optional[float] = None
    - num_virtual_tokens: Optional[int] = None
    - adapter_dim: Optional[int] = None
```

As you can see there are a number of parameters we can pass in.

Required are the name of the model you wish to p-tune, the human-readable name of the customized model, and the ID of the training dataset. If you do not supply a validation dataset file, NeMo Service will take care to split off 10% of your training data to use for validation.

In addition to these required fields, you can also override the training hyperparamter defaults. In the case of p-tuning we should consider the following hyperparameters.

### Batch Size

The default value is 8. A value of 16 is often recommended as a good place to start experimentation.

### Epochs

The default value is 50. Depending on the job a single epoch can do the trick, but typically 3 to 10 epochs is better. NeMo Service will end training early if after 10 epochs the validation error has not improved, so, unless you're in a tremendous rush, you can just set this to 50 and see when the job completes early.

Depending on NeMo Services's load, your p-tuning job may begin immediately or may be queued up for up to 5 minutes or so. It depends on your dataset size, the length of your generation, the batch size, and the size of your model (smaller is faster), but typically you can expect ~5-10 minutes per epoch.

### Learning Rate

Set to a good default of .0001, you can adjust this if you wish.

### Number of Virtual Tokens

Specific to p-tuning, this is where you configure how many virtual tokens to train. The default is 50, but for smaller models you might consider beginning your experiements with a smaller value like 10. Experimentation and observation of empirical results in the best way to obtain the optimal value, but we have found the default value of 50 to be an excellent value to start with.

---

## Run P-tuning

We've already run the p-tuning jobs for this task, so you don't need to actually run the jobs yourself, but for your reference, the following cells show how we ran the jobs that resulted in the models we will be using in the next notebook. In each case we trained for 3 epochs with the default values for batch size (8), number of virutal tokens (50) and learning rate (.0001). We ran these jobs concurrently and on average each took 20 minutes to train over 3 epochs.

```python
response_43b = conn.create_customization(
    model=PtuneableModels.gpt43b.value,
    name='pubmedqa-43b-token-50-batch-8-epochs-3',
    description="P-tuning for custom pubmedqa model.",
    batch_size=8,
    num_virtual_tokens=50,
    training_dataset_file_id=training_dataset_file_id,   
    validation_dataset_file_id=validate_dataset_file_id,   
    epochs=3
)
```

```python
response_20b = conn.create_customization(
    model=PtuneableModels.gpt20b.value,
    name='pubmedqa-20b-token-50-batch-8-epochs-3',
    description="P-tuning for custom pubmedqa model.",
    batch_size=8,
    num_virtual_tokens=50,
    validation_dataset_file_id=validate_dataset_file_id,   
    training_dataset_file_id=training_dataset_file_id,   
    epochs=3
)
```

```python
response_8b = conn.create_customization(
    model=PtuneableModels.gpt8b.value,
    name='pubmedqa-8b-token-50-batch-8-epochs-3',
    description="P-tuning for custom pubmedqa model.",
    batch_size=8,
    num_virtual_tokens=50,
    validation_dataset_file_id=validate_dataset_file_id,   
    training_dataset_file_id=training_dataset_file_id,   
    epochs=3
)
```